In [1]:
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import  Chroma
from langchain.document_loaders import JSONLoader
import chromadb

In [2]:

model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


/home/20015587/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/20015587/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-17 23:57:49.708212: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computa

In [3]:
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["author"] = record.get("author")
    metadata["tags"] = ' '.join(record.get("tags"))

    return metadata

In [4]:
dir_path="./data/quotes.json"
chroma_instance = chromadb.Client()
doc_loader = JSONLoader(dir_path,".[]",content_key="text",metadata_func=metadata_func)
docs = doc_loader.load()

In [5]:
docs[0]

Document(page_content='“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', metadata={'source': '/home/20015587/Documents/IA/hugging-face/data/quotes.json', 'seq_num': 1, 'author': 'Albert Einstein', 'tags': 'change deep-thoughts thinking world'})

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
split_docs = splitter.split_documents(docs)
db = Chroma.from_documents(documents=split_docs, embedding=model)

In [7]:
query="One good thing about music Albert"
retrieved_docs = db.similarity_search(query,k=2)
retrieved_docs

[Document(page_content='“One good thing about music, when it hits you, you feel no pain.”', metadata={'author': 'Bob Marley', 'seq_num': 30, 'source': '/home/20015587/Documents/IA/hugging-face/data/quotes.json', 'tags': 'music'}),
 Document(page_content='“If I were not a physicist, I would probably be a musician. I often think in music. I live my daydreams in music. I see my life in terms of music.”', metadata={'author': 'Albert Einstein', 'seq_num': 54, 'source': '/home/20015587/Documents/IA/hugging-face/data/quotes.json', 'tags': 'music'})]

In [8]:
from flask import Flask, request, jsonify, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/search')
def search():
    query = request.args.get('q', '')
    if not query:
        return jsonify([])

    # Perform search in Elasticsearch
    response =db.similarity_search(query,k=3)
   
    return jsonify(response)

def run_app():
    app.run(host='0.0.0.0', port=5000)



In [9]:
from threading import Thread
flask_thread = Thread(target=run_app)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.44:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Jun/2024 23:57:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2024 23:58:15] "GET /search?q=sedd HTTP/1.1" 404 -
